In [70]:
import pandas as pd

In [71]:
afile = '../writeup/tables/adult.all.eur.assoc.csv'
df = pd.read_csv(afile, sep=',').dropna()
df.head()

,SNP,gene,A1_plink,A2_plink,P_plink,OR_plink,A1_snptest,A2_snptest,P_snptest,OR_snptest,cases_maf,controls_maf,A1_adult,A2_adult,PVAL_adult,OR_adult,A1_vcf,A2_vcf,eff
0,5:71693899,RP11-389C8.1-YBX1P5,T,C,0.006132,2.0770,T,C,0.003501,0.481481,0.346154,0.203125,A,G,7.160000e-11,0.931405,C,T,PR;ANN=T|intergenic_region|MODIFIER|RP11-389C8...
1,11:76299194,RP11-672A2.7,G,T,0.002038,0.4865,T,G,0.003582,0.486600,0.362681,0.539062,A,C,8.710000e-48,1.156401,T,G,PR;ANN=G|upstream_gene_variant|MODIFIER|RP11-6...
2,1:197631141,DENND1B,A,G,0.008931,2.0870,A,G,0.007180,0.479233,0.302203,0.171875,A,G,1.260000e-15,1.101036,G,A,PR;ANN=A|upstream_gene_variant|MODIFIER|DENND1...
3,2:28614794,FOSL2,T,C,0.012960,0.5603,T,C,0.014634,1.794090,0.372368,0.515602,A,G,1.070000e-16,0.904798,C,T,PR;ANN=T|upstream_gene_variant|MODIFIER|FOSL2|...
4,9:139266405,CARD9,A,G,0.029340,1.6670,A,G,0.015487,0.599440,0.500000,0.374781,A,G,1.710000e-53,1.167542,G,A,PR;ANN=A|synonymous_variant|LOW|CARD9|ENSG0000...


In [72]:
df.loc[:, 'OR_snptest'] = df.apply(lambda row: 1/row['OR_snptest'], axis=1)

In [73]:
df.head()

,SNP,gene,A1_plink,A2_plink,P_plink,OR_plink,A1_snptest,A2_snptest,P_snptest,OR_snptest,cases_maf,controls_maf,A1_adult,A2_adult,PVAL_adult,OR_adult,A1_vcf,A2_vcf,eff
0,5:71693899,RP11-389C8.1-YBX1P5,T,C,0.006132,2.0770,T,C,0.003501,2.076925,0.346154,0.203125,A,G,7.160000e-11,0.931405,C,T,PR;ANN=T|intergenic_region|MODIFIER|RP11-389C8...
1,11:76299194,RP11-672A2.7,G,T,0.002038,0.4865,T,G,0.003582,2.055076,0.362681,0.539062,A,C,8.710000e-48,1.156401,T,G,PR;ANN=G|upstream_gene_variant|MODIFIER|RP11-6...
2,1:197631141,DENND1B,A,G,0.008931,2.0870,A,G,0.007180,2.086668,0.302203,0.171875,A,G,1.260000e-15,1.101036,G,A,PR;ANN=A|upstream_gene_variant|MODIFIER|DENND1...
3,2:28614794,FOSL2,T,C,0.012960,0.5603,T,C,0.014634,0.557386,0.372368,0.515602,A,G,1.070000e-16,0.904798,C,T,PR;ANN=T|upstream_gene_variant|MODIFIER|FOSL2|...
4,9:139266405,CARD9,A,G,0.029340,1.6670,A,G,0.015487,1.668224,0.500000,0.374781,A,G,1.710000e-53,1.167542,G,A,PR;ANN=A|synonymous_variant|LOW|CARD9|ENSG0000...


In [74]:
def mk_ped_alleles(row, ped_a1, ped_a2, ped_or):
    a1, a2, oR = '', '', ''
    comp = {'A':'T', 'T':'A', 'C':'G', 'G':'C'}
    nuc1, nuc2 = row[ped_a1], row[ped_a2]
    if row['A1_adult'] == nuc1 and row['A2_adult'] == nuc2:
        a1, a2, oR = nuc1, nuc2, row[ped_or]
    elif row['A1_adult'] == nuc2 and row['A2_adult'] == nuc1:
        a1, a2, oR = nuc2, nuc1, 1/row[ped_or]
    
    if not a1:
        # test complement
        nuc1, nuc2 = comp[row[ped_a1]], comp[row[ped_a2]]
        if row['A1_adult'] == nuc1 and row['A2_adult'] == nuc2:
            a1, a2, oR = nuc1, nuc2, row[ped_or]
        elif row['A1_adult'] == nuc2 and row['A2_adult'] == nuc1:
            a1, a2, oR = nuc2, nuc1, 1/row[ped_or]
        else:
            print(row)
            print(row['A1_adult'], nuc1, row['A2_adult'], nuc2)
            i =1/0
        
    dat = {'a1':a1, 'a2':a2, 'or':oR}                    
    return pd.Series(dat, index=['a1', 'a2', 'or'])  

def or_agree(row):
    """Does OR agree between pediatric snptest/plink and adult?"""
    if row['OR_snptest'] > 1 and row['OR_plink'] < 1:
        return 'strange plink/snptest mismatch'
        #i = 1/0
    if row['OR_snptest'] < 1 and row['OR_plink'] > 1:
        return 'strange plink/snptest mismatch'
        #print(row)
        #i = 1/0
        
    if row['OR_snptest'] > 1 and row['OR_adult'] > 1:
        return 'match'
    if row['OR_snptest'] < 1 and row['OR_adult'] < 1:
        return 'match'
    
    return 'mismatch'

In [75]:
df[['a1', 'a2', 'or']] = df.apply(lambda row: mk_ped_alleles(row, 'A1_plink', 'A2_plink', 'OR_plink'), axis=1)
df.loc[:, 'A1_plink'] = df['a1']
df.loc[:, 'A2_plink'] = df['a2']
df.loc[:, 'OR_plink'] = df['or']
df = df.drop(['a1', 'a2', 'or'], axis=1)

df[['a1', 'a2', 'or']] = df.apply(lambda row: mk_ped_alleles(row, 'A1_snptest', 'A2_snptest', 'OR_snptest'), axis=1)
df.loc[:, 'A1_snptest'] = df['a1']
df.loc[:, 'A2_snptest'] = df['a2']
df.loc[:, 'OR_snptest'] = df['or']
df = df.drop(['a1', 'a2', 'or'], axis=1)
df.loc[:, 'OR_agree'] = df.apply(or_agree, axis=1)

In [76]:
df.head()

,SNP,gene,A1_plink,A2_plink,P_plink,OR_plink,A1_snptest,A2_snptest,P_snptest,OR_snptest,cases_maf,controls_maf,A1_adult,A2_adult,PVAL_adult,OR_adult,A1_vcf,A2_vcf,eff,OR_agree
0,5:71693899,RP11-389C8.1-YBX1P5,A,G,0.006132,2.077000,A,G,0.003501,2.076925,0.346154,0.203125,A,G,7.160000e-11,0.931405,C,T,PR;ANN=T|intergenic_region|MODIFIER|RP11-389C8...,mismatch
1,11:76299194,RP11-672A2.7,A,C,0.002038,2.055498,A,C,0.003582,2.055076,0.362681,0.539062,A,C,8.710000e-48,1.156401,T,G,PR;ANN=G|upstream_gene_variant|MODIFIER|RP11-6...,match
2,1:197631141,DENND1B,A,G,0.008931,2.087000,A,G,0.007180,2.086668,0.302203,0.171875,A,G,1.260000e-15,1.101036,G,A,PR;ANN=A|upstream_gene_variant|MODIFIER|DENND1...,match
3,2:28614794,FOSL2,A,G,0.012960,0.560300,A,G,0.014634,0.557386,0.372368,0.515602,A,G,1.070000e-16,0.904798,C,T,PR;ANN=T|upstream_gene_variant|MODIFIER|FOSL2|...,match
4,9:139266405,CARD9,A,G,0.029340,1.667000,A,G,0.015487,1.668224,0.500000,0.374781,A,G,1.710000e-53,1.167542,G,A,PR;ANN=A|synonymous_variant|LOW|CARD9|ENSG0000...,match


In [77]:
df.groupby('OR_agree').size()

OR_agree
match                             142
mismatch                           86
strange plink/snptest mismatch      1
dtype: int64